In [5]:
import pandas as pd
from sklearn import preprocessing
import sklearn

%pylab inline
# display in expanded form. Thanks stackoverflow
pd.set_option('display.float_format', lambda x: '%.1f' % x)

data = pd.read_csv('KAGGLEairlinefleetsfleetdata.csv', sep=',')

data.head()

Populating the interactive namespace from numpy and matplotlib


,Parent Airline,Airline,Aircraft Type,Current,Future,Historic,Total,Orders,Unit Cost,Total Cost (Current),Average Age
0,Aegean Airlines,Aegean Airlines,Airbus A319,1.0,nan,3.0,4.0,nan,$90,$90,11.6
1,Aegean Airlines,Olympic Air,Airbus A319,nan,nan,8.0,8.0,nan,$90,$0,nan
2,Aegean Airlines,Aegean Airlines,Airbus A320,38.0,nan,3.0,41.0,nan,$98,"$3,724",7.5
3,Aegean Airlines,Olympic Air,Airbus A320,nan,nan,9.0,9.0,nan,$98,$0,nan
4,Aegean Airlines,Aegean Airlines,Airbus A321,8.0,nan,nan,8.0,nan,$115,$919,10.3


data from https://www.kaggle.com/traceyvanp/airlinefleet

if current = NaN, assume 0. Basically means dataset row is blank, which means 0 / no data - done

If current = 0/NaN, then current cost is 0, fleet age is 0, etc. - done

remove $ signs from unit and total cost. - done

Concatenate future and orders. They are the same thing but from different sources... Future form planespotter.net, orders from wikipedia. Use only wikipedia. - done

Change total and unit cost to be actual millions. Not really necessary, because its going to be squashed into the (0,1) range anyway... - Done

normalize & standardize.

Replace Parent Airline, Airplane, Aircraft Type with one hot arrays. Newest dev version of sklearn has onehot encoder that can do strings: pip3 install git+git://github.com/scikit-learn/scikit-learn.git

NOTE: Total cost is current * unit (ish)... It is already a feature interaction

In [6]:
# replace all NaN with 0. The only column that this is definitely correct for is Average Age
data = data.fillna(0)

# Use wikipedia orders instead of planespotter.net. Made this decision by looking at file in excel
data = data.drop("Future", 1)

# get rid of $ signs, and convert from $M to $ (eg $1 becomes 1000000)
data = data.replace('\$+','', regex=True).replace('\,+','', regex=True)
    # I don't know what the regex really does, but it works!

def expandMillions(input):
    return float(input) * 1000000.0

data["Unit Cost"] = data["Unit Cost"].apply(expandMillions)
data["Total Cost (Current)"] = data["Total Cost (Current)"].apply(expandMillions)
# can I do the above two lines in one line?

# basically onehot:
#aircraft_type_encoder.classes_ to access an array of class names. inverse_transform to undo encoding
parent_airline_encoder = preprocessing.LabelBinarizer()
parent_airline_encoded = parent_airline_encoder.fit_transform(data['Parent Airline'])

airline_encoder = preprocessing.LabelBinarizer()
airline_encoded = airline_encoder.fit_transform(data['Airline'])

aircraft_type_encoder = preprocessing.LabelBinarizer()
aircraft_type_encoded = aircraft_type_encoder.fit_transform(data['Aircraft Type'])

data = data.drop(['Parent Airline', 'Airline', 'Aircraft Type'], axis=1)

In [7]:
'''
create a column in the data object for each airline/parent airline/aircraft type
the ith index of each row needs to be added to column i of the dataset
'''
def onehot_to_dataset(dataset, onehot):
    columns=[]
    
    for i, row in enumerate(onehot):
        for j, column in enumerate(row):
            if not i:
                columns.append([])
            columns[j].append(column)
    return columns

airline_encoded = onehot_to_dataset(data, airline_encoded)
parent_airline_encoded = onehot_to_dataset(data, parent_airline_encoded)
aircraft_type_encoded = onehot_to_dataset(data, aircraft_type_encoded)

def add_to_dataset(dataset, columns, base_col_name):
    for i, column in enumerate(columns):
        dataset.insert(0, base_col_name + str(i)+"?", column)

add_to_dataset(data, airline_encoded, "Airline")
add_to_dataset(data, parent_airline_encoded, "Parent_Airline")
add_to_dataset(data, aircraft_type_encoded, "Aircraft_Type")

data.head()

,Aircraft_Type100?,Aircraft_Type99?,Aircraft_Type98?,Aircraft_Type97?,Aircraft_Type96?,Aircraft_Type95?,Aircraft_Type94?,Aircraft_Type93?,Aircraft_Type92?,Aircraft_Type91?,...,Airline2?,Airline1?,Airline0?,Current,Historic,Total,Orders,Unit Cost,Total Cost (Current),Average Age
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1.0,3.0,4.0,0.0,90000000.0,90000000.0,11.6
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,8.0,8.0,0.0,90000000.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,38.0,3.0,41.0,0.0,98000000.0,3724000000.0,7.5
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,9.0,9.0,0.0,98000000.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,8.0,0.0,8.0,0.0,115000000.0,919000000.0,10.3


In [9]:
# now just need to scale from 0 to 1
def scale0to1(col):
    # first find max and min
    max = 0
    min = 0
    for i, val in enumerate(col):
        if not i:
            max = val
            min = val
        if val > max:
            max = val
        if val < min:
            min = val
            
    range = max - min
    
    # then scale each value
    scaled = []
    for i, val in enumerate(col):
        scaled.append((val - min) / range)
    return scaled # in a real application, the max and min should also be returned, so that they can be used to later interpret predictions

data['Current'] = scale0to1(data['Current'])
data['Historic'] = scale0to1(data['Historic'])
data['Total'] = scale0to1(data['Total'])
data['Orders'] = scale0to1(data['Orders'])
data['Unit Cost'] = scale0to1(data['Unit Cost'])
data['Total Cost (Current)'] = scale0to1(data['Total Cost (Current)'])
data['Average Age'] = scale0to1(data['Average Age'])

data.head()

,Aircraft_Type100?,Aircraft_Type99?,Aircraft_Type98?,Aircraft_Type97?,Aircraft_Type96?,Aircraft_Type95?,Aircraft_Type94?,Aircraft_Type93?,Aircraft_Type92?,Aircraft_Type91?,...,Airline2?,Airline1?,Airline0?,Current,Historic,Total,Orders,Unit Cost,Total Cost (Current),Average Age
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0.0,0.0,0.0,0.0,0.2,0.0,0.3
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0.2,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0.1,0.0,0.0,0.0,0.2,0.1,0.2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0.2,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0.0,0.0,0.0,0.0,0.3,0.0,0.3
